In [3]:
import requests
import json
import logging

logging.basicConfig(level=logging.DEBUG)

def usr_aut(gateway, username="", password=""):
    try:
        payload = {
            "token": "",
            "action": "usr_aut",
            "parameters": {
                "user": username,
                "password": password
                }
            }
    except Exception as e:
        logging.error(e)
        return None
    auth = requests.post(url = gateway, data = json.dumps(payload))
    if auth.status_code == 200:
        try:
            token = json.loads(auth.content)['result'][0]['serviceresult']['response']
            return token
        except Exception as e:
            logging.error(str(e))
            return None
    else:
        logging.error('Error connecting to {}, response code is '.format(gateway) + str(auth))
        return None
    
def frx_tensorflow_train(token, data_object, model, organization):
    # NOTE: Ignoring these columns:
    # ['BillingAddress' 'BillingCity' 'BillingState' 'BillingCountry' 'BillingPostalCode' 'InvoiceLineId']
    data = {
            'model_type': model,
            'organization': organization,
            'dataset': data_object,
            'index_schema': [
                {
                    'column_name': 'InvoiceDate',
                    'dtype': 'datetime',
                    'format': '' # alternatively can pass in the format for datetime
                }
            ],
            'context_schema': [
                {
                    'column_name': 'InvoiceId',
                    'dtype': 'int',
                    'format': 'categorcial'
                },
                {
                    'column_name': 'CustomerId',
                    'dtype': 'int',
                    'format': 'categorical'
                },
                {
                    'column_name': 'TrackId',
                    'dtype': 'int',
                    'format': 'categorical'
                }
            ],
            'target_schema': [
                {
                    'column_name': 'Total',
                    'dtype': 'float',
                    'format': 'numeric'
                },
                {
                    'column_name': 'UnitPrice',
                    'dtype': 'float',
                    'format': 'numeric'
                },
                {
                    'column_name': 'Quantity',
                    'dtype': 'int',
                    'format': 'numeric'
                }
            ],
        }
    data = json.dumps(data)
    try:
        payload = {
            "token": token,
            "action": "tensorflow_train",
            "parameters": {
                "data": data
                }
            }
    except Exception as e:
        logging.error(e)
        return None
    auth = requests.post(url = gateway, data = json.dumps(payload), timeout=300)
    if auth.status_code == 200:
        try:
            token = json.loads(auth.content)['result'][0]['serviceresult']['response']
            return token
        except Exception as e:
            logging.error(str(e))
            return None
    else:
        logging.error('Error connecting to {}, response code is '.format(gateway) + str(auth))
        return None
gateway = 'https://fraxses-us.eastus.cloudapp.azure.com/api/gateway'

token = usr_aut(gateway, username="ryan_walden", password="ryan_walden")['records'][0]['tok']
print(token)

resp = frx_tensorflow_train(
    token, 
    data_object="odc_ap_demo_020", 
    model="AnomalyDetector", 
    organization='abc'
)
print(resp)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): fraxses-us.eastus.cloudapp.azure.com:443
DEBUG:urllib3.connectionpool:https://fraxses-us.eastus.cloudapp.azure.com:443 "POST /api/gateway HTTP/1.1" 200 493
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): fraxses-us.eastus.cloudapp.azure.com:443


FCC234D3-DD48-44E4-B81B-7A8D3073DB2A


DEBUG:urllib3.connectionpool:https://fraxses-us.eastus.cloudapp.azure.com:443 "POST /api/gateway HTTP/1.1" 200 315


Error tuning the model, No module named 'utils'


https://docs.ray.io/en/master/installation.html?highlight=docker#launch-ray-in-docker
- name: dshm
      emptyDir:
        medium: Memory

- name: dshm
  mountPath: /dev/shm
